# Environmental Solutions Lab: Adaptive Infrastructure
## Tasks I - III: Data preparation
Intelligent Earth Center for Doctoral Training <br>
Friday 13th December 2024 <br>

Instructors:
* <alison.peard@ouce.ox.ac.uk><br>
* <yu.mo@chch.ox.ac.uk>


In [1]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

WD = os.path.join(os.getcwd(), "data")

| Dataset | Download URL |
| ------- | ------------ |
| IBTrACs | https://www.ncei.noaa.gov/products/international-best-track-archive |
| IBTrACs metadata (a) | https://climada-python.readthedocs.io/en/stable/_modules/climada/hazard/tc_tracks.html |
| IBTrACs metadata (b) | https://doi.org/10.1175/2009BAMS2755.1 |
| Blackout | https://blackmarble.gsfc.nasa.gov |
| Natural Earth Cultural| https://www.naturalearthdata.com/downloads/10m-cultural-vectors |


### Task I: Clean IBTrACS data for storm characteristics input
The IBTrACS (International Best Track Archive for Climate Stewardship) is a comprehensive dataset for tropical cyclones. You will clean and format this dataset for input into predictive models. The steps include:
* Formatting: Review the source file for inconsistencies or formatting errors. Ensure the data is in a consistent and analysable format.
* Standardizing Records Across Agencies: IBTrACS consolidates data from multiple agencies. Harmonize these variables to ensure uniformity, such as converting all wind speed measurements to the same standard.


In [2]:
IBTRACS = "ibtracs.since1980.list.v04r00.csv"
AGENCIES = "wind_factors.csv"

COLUMNS_IN = ["SID", "SEASON", "NUMBER","BASIN","NAME","NATURE","ISO_TIME",
           "LAT","LON","WMO_WIND","WMO_PRES","WMO_AGENCY",
           "DIST2LAND","LANDFALL",'USA_WIND','USA_RMW','REUNION_WIND','REUNION_RMW',
           'BOM_WIND',"BOM_RMW","STORM_SPEED"]

COLUMNS_OUT = ["SID","SEASON","NUMBER","BASIN","NAME","NATURE","ISO_TIME",
               "LAT","LON","WMO_WIND","WMO_PRES","STORM_SPEED","DIST2LAND",
               "LANDFALL"]

In [3]:
# load data
ibtracs = pd.read_csv(os.path.join(WD, IBTRACS), 
skiprows=[1]) # second row is units
ibtracs.head()

/tmp/ipykernel_564/2577726071.py:2: DtypeWarning: Columns (19,20,161,162) have mixed types. Specify dtype option on import or set low_memory=False.
  ibtracs = pd.read_csv(os.path.join(WD, IBTRACS),


,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,...,BOM_GUST_PER,REUNION_GUST,REUNION_GUST_PER,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR
0,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 00:00:00,TS,-12.5000,172.500,...,,,,,,,,,6,351
1,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 03:00:00,TS,-12.1927,172.441,...,,,,,,,,,6,351
2,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 06:00:00,TS,-11.9144,172.412,...,,,,,,,,,5,358
3,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 09:00:00,TS,-11.6863,172.435,...,,,,,,,,,4,12
4,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 12:00:00,TS,-11.5000,172.500,...,,,,,,,,,4,22


In [4]:
# check longitude range
print("Longitude range:", ibtracs['LON'].min(), ibtracs['LON'].max()) # ibtracs['LON'].max()
print("Latitude range:", ibtracs['LAT'].min(), ibtracs['LAT'].max())

Longitude range: -179.8 266.9
Latitude range: -68.5 70.7


The longitude range looks strange, longitude is usually in (-180, 180] or [0, 360] while latitude is usually in [-90, 90] or [0, 180]. We solve this by simply subtracting 360 from any longitudes greater than 180.

In [5]:
def fix_longitude(lon):
    if lon > 180:
        return lon - 360
    return lon

ibtracs['LON'] = ibtracs['LON'].apply(fix_longitude)
print("Longitude range:", ibtracs['LON'].min(), ibtracs['LON'].max()) 

Longitude range: -179.998 180.0


In [6]:
# there are some blank rows in WMO_WIND
ibtracs = ibtracs[ibtracs["WMO_WIND"] != ' '].copy()
ibtracs['WMO_WIND'] = ibtracs['WMO_WIND'].astype(float)
ibtracs = ibtracs[COLUMNS_IN].copy()

The World Meteorolical Organisation (WMO) defines the _maximum sustained wind_ as the 10-minute average wind speed 10m above ground level ($U_{10}$).

Despite this, many organisations average over one-minute or three-minute periods. [Knapp et al (2010)](https://journals.ametsoc.org/view/journals/mwre/138/4/2009mwr3123.1.xml) provide a table of shift and scale factors to convert different agencies to the standard 10-minute maximum sustained wind.

We will use a dataset of shift and scale factors to standardise our wind records.

In [7]:
wind_factors = pd.read_csv(os.path.join(WD, AGENCIES), index_col=0)
wind_factors.head()

,scale,shift
agency,,
usa,1.00,0.0
tokyo,0.60,23.3
newdelhi,1.00,0.0
reunion,0.88,0.0
bom,0.88,0.0


In [8]:
def convert_winds(row:pd.Series) -> float:
    agency = row['WMO_AGENCY']
    scale = wind_factors.loc[agency, 'scale']
    shift = wind_factors.loc[agency, 'shift']
    new_wind = row['WMO_WIND'] * scale + shift
    return new_wind

ibtracs['WMO_WIND'] = ibtracs.apply(convert_winds, axis=1)

In [9]:
ibtracs[COLUMNS_OUT].to_csv(os.path.join(WD, "ibtracs_formatted"), index=False)

### Task II: Get storm information for blackout events
The blackout dataset, derived from sources like the Black Marble project, captures key details about power outages, including duration, affected customers, and geographic locations. To integrate storm-related information into this dataset, complete the following steps:
* Matching records: match the IBTrACS data with the blackout data by a common identifier, i.e., storm ID. 
* Extract storm information: For each matched blackout event, extract relevant storm characteristics from the IBTrACS dataset, such as wind speed, pressure, and speed.


In [10]:
BLACKOUT = "blackout.csv"
IBTRACS = "ibtracs_formatted"
JOINCOLS = ["NAME", "SEASON", "WMO_WIND", "STORM_SPEED", "LON", "LAT"]

blackout = pd.read_csv(os.path.join(WD, BLACKOUT))
ibtracs = pd.read_csv(os.path.join(WD, IBTRACS))
ibtracs = ibtracs.set_index("SID")
blackout.head()

/tmp/ipykernel_564/3057917379.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  ibtracs = pd.read_csv(os.path.join(WD, IBTRACS))


,sid,urbanLevel,ID,lon,lat,pxlmx,before_mean,before_sd,duration,durationWithMissing
0,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5
1,2012086N10116_2012033106,HDC,30,106.382987,10.218337,1,2.102590,0.637378,12,15
2,2012086N10116_2012033106,LDC,106,106.499763,10.397988,10,30.927924,14.254545,2,5
3,2012086N10116_2012033106,LDC,112,106.607566,11.588266,2,0.621593,0.195296,6,12
4,2012086N10116_2012033106,LDC,123,106.742314,11.206483,1,0.661250,0.204993,3,4


In [11]:
# Process times from sid column
blackout['landing_time'] = blackout['sid'].str.split("_").str[1]
blackout['landing_year'] = blackout['landing_time'].str[:4]
blackout['landing_month'] = blackout['landing_time'].str[4:6]
blackout['landing_day'] = blackout['landing_time'].str[6:8]
blackout['landing_hour'] = blackout['landing_time'].str[8:10]
blackout.head()


,sid,urbanLevel,ID,lon,lat,pxlmx,before_mean,before_sd,duration,durationWithMissing,landing_time,landing_year,landing_month,landing_day,landing_hour
0,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5,2012033106,2012,03,31,06
1,2012086N10116_2012033106,HDC,30,106.382987,10.218337,1,2.102590,0.637378,12,15,2012033106,2012,03,31,06
2,2012086N10116_2012033106,LDC,106,106.499763,10.397988,10,30.927924,14.254545,2,5,2012033106,2012,03,31,06
3,2012086N10116_2012033106,LDC,112,106.607566,11.588266,2,0.621593,0.195296,6,12,2012033106,2012,03,31,06
4,2012086N10116_2012033106,LDC,123,106.742314,11.206483,1,0.661250,0.204993,3,4,2012033106,2012,03,31,06


In [12]:
# now match based-on SID and extract name, season
blackout['SID'] = blackout['sid'].str.split("_").str[0]
blackout = blackout.set_index("SID")

In [13]:
blackout = blackout.join(ibtracs[JOINCOLS])
blackout = blackout.rename(columns={"NAME": "storm", "SEASON": "season"})

## Notes
There are a number of ways to calculate the distances between two geospatial locations. The simplest is to use the `haversine` library. If this is not available, the haversine distance can be computed manually. This can also be achieved with `geopandas`.

In [14]:
def calculate_distances_haversine(blackout):
    from haversine import haversine

    def haversine_dist(coords):
        lat1, lon1, lat2, lon2 = coords
        return haversine((lat1, lon1), (lat2, lon2), unit='km')
    
    blackout = blackout.copy()
    blackout['distToTrack'] = blackout[['lat', 'lon', 'LAT', 'LON']].apply(haversine_dist, axis=1)
    blackout['distToTrack'] = blackout['distToTrack'] * 1000 # convert to meters
    return blackout


def calculate_distances_manual(blackout):
    from math import radians, cos, sin, asin, sqrt

    def haversine_dist(coords):
        """https://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points"""
        lat1, lon1, lat2, lon2 = coords
        R = 6372.8 # this is in km.  For Earth radius in kilometers use 3959.87433 miles

        dLat = radians(lat2 - lat1)
        dLon = radians(lon2 - lon1)
        lat1 = radians(lat1)
        lat2 = radians(lat2)

        a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
        c = 2*asin(sqrt(a))

        return R * c
    
    blackout = blackout.copy()
    blackout['distToTrack'] = blackout[['lat', 'lon', 'LAT', 'LON']].apply(haversine_dist, axis=1)
    blackout['distToTrack'] = blackout['distToTrack'] * 1000 # convert to meters
    return blackout


def calculate_distances_with_geopandas(blackout):
    import geopandas as gpd

    columns = list(blackout.columns)
    blackout = gpd.GeoDataFrame(blackout)
    blackout['points_blackout'] = gpd.GeoSeries.from_xy(blackout['lon'], blackout['lat']).set_crs(4326)
    blackout['points_ibtracs'] = gpd.GeoSeries.from_xy(blackout['LON'], blackout['LAT']).set_crs(4326)

    # switch to Pseudo Mercator (in metres)
    blackout['points_blackout'] = blackout['points_blackout'].to_crs(3857)
    blackout['points_ibtracs'] = blackout['points_ibtracs'].to_crs(3857)

    blackout['distToTrack'] = blackout['points_blackout'].distance(blackout['points_ibtracs'])
    blackout['distToTrack'] = blackout['distToTrack']

    # geopandas doesn't handle NaNs well
    nan_mask = blackout[['lat', 'lon', 'LAT', 'LON']].isnull().any(axis=1)
    blackout.loc[nan_mask, 'distToTrack'] = pd.NA

    return blackout[columns + ['distToTrack']]

# play around with these
distance_func = calculate_distances_with_geopandas

In [15]:
# get distances
blackout = distance_func(blackout)
blackout['distToTrack'].describe()


count    7.399540e+05
mean     1.598146e+06
std      2.112443e+06
min      6.023085e+02
25%      6.007024e+05
50%      1.220173e+06
75%      2.095459e+06
max      3.988226e+07
Name: distToTrack, dtype: float64

In [16]:
# aggregate over storm: maxwind, mean speed, min dist to track
storm_characteristics = blackout.groupby("SID").agg(
    windmax=("WMO_WIND", "max"),
    speed=("STORM_SPEED", "mean"),
    distToTrack=("distToTrack", "min")
)

storm_characteristics.head()

,windmax,speed,distToTrack
SID,,,
2012086N10116,47.3,4.071429,8023.356758
2012142N09262,100.0,7.36,149110.786874
2012147N30284,60.0,12.636364,5708.153338
2012162N06150,83.3,19.222222,29484.764862
2012166N09269,95.0,9.777778,34934.634292


In [17]:
blackout[['windmax', 'speed', 'distToTrack']] = storm_characteristics
blackout

,sid,urbanLevel,ID,lon,lat,pxlmx,before_mean,before_sd,duration,durationWithMissing,...,landing_hour,storm,season,WMO_WIND,STORM_SPEED,LON,LAT,distToTrack,windmax,speed
SID,,,,,,,,,,,,,,,,,,,,,
2012086N10116,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5,...,06,PAKHAR,2012.0,44.3,2,111.875,9.675,8023.356758,47.3,4.071429
2012086N10116,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5,...,06,PAKHAR,2012.0,44.3,2,111.650,9.675,8023.356758,47.3,4.071429
2012086N10116,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5,...,06,PAKHAR,2012.0,47.3,3,111.375,9.675,8023.356758,47.3,4.071429
2012086N10116,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5,...,06,PAKHAR,2012.0,47.3,3,111.075,9.700,8023.356758,47.3,4.071429
2012086N10116,2012086N10116_2012033106,HDC,19,105.978746,9.589516,1,1.793363,0.511346,3,5,...,06,PAKHAR,2012.0,47.3,3,110.775,9.725,8023.356758,47.3,4.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021346N05145,2021346N05145_2021121906,RUR,30,109.167765,14.170924,1,0.962038,0.212420,5,5,...,06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021358S09130,2021358S09130_2022010512,LDC,2,152.893261,-25.310039,1,3.871187,0.723764,1,4,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021358S09130,2021358S09130_2022010512,LDC,4,153.050466,-26.388012,1,3.028112,0.586223,1,4,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# save processed blackout data
blackout = blackout.dropna(subset=['storm'])
blackout.to_csv(os.path.join(WD, "blackout_with_storms.csv"), index=False)

### Task III: Match socioeconomic data with blackout data
Socio-economic factors such as population density, income, and urbanization levels are critical for understanding the context of power outages. To incorporate this information into the dataset, follow these steps:
* Geographic matching: Use location identifiers, i.e., latitude and longitude, to link socio-economic data with the blackout dataset.
* Handle missing socio-economic data: Identify missing socio-economic values in the matched dataset. Apply approximation techniques to fill in the empty data.


In [19]:
# reload the blackout data as a geospatial dataframe this time
blackout = pd.read_csv(os.path.join(WD, "blackout_with_storms.csv"))
blackout = gpd.GeoDataFrame(blackout, geometry=gpd.points_from_xy(blackout.lon, blackout.lat))
blackout = blackout.set_crs(4326)

#### Socioeconomic data
Versions of geopandas ≤ 1.0 have a `geopandas.datasets` module which allows us to import Natural Earth socioeconomic data directly to Python. Newer versions [have deprecated this module](https://github.com/geopandas/geopandas/issues/2751) but the country-level data (Admin 0 - Countries) can be downloaded from the [Natural Earth downloads page](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/).

In [20]:
# method 1
SOCIOFILE = "ne_110m_admin_0_countries"
SOCIOCOLS = ['pop_est', 'continent', 'name', 'iso3', 'gdp_md_est', 'income', 'geometry']

socio = gpd.read_file(os.path.join(WD, SOCIOFILE))
print(list(socio.columns))
print(list(socio['INCOME_GRP'].unique()))

['featurecla', 'scalerank', 'LABELRANK', 'SOVEREIGNT', 'SOV_A3', 'ADM0_DIF', 'LEVEL', 'TYPE', 'TLC', 'ADMIN', 'ADM0_A3', 'GEOU_DIF', 'GEOUNIT', 'GU_A3', 'SU_DIF', 'SUBUNIT', 'SU_A3', 'BRK_DIFF', 'NAME', 'NAME_LONG', 'BRK_A3', 'BRK_NAME', 'BRK_GROUP', 'ABBREV', 'POSTAL', 'FORMAL_EN', 'FORMAL_FR', 'NAME_CIAWF', 'NOTE_ADM0', 'NOTE_BRK', 'NAME_SORT', 'NAME_ALT', 'MAPCOLOR7', 'MAPCOLOR8', 'MAPCOLOR9', 'MAPCOLOR13', 'POP_EST', 'POP_RANK', 'POP_YEAR', 'GDP_MD', 'GDP_YEAR', 'ECONOMY', 'INCOME_GRP', 'FIPS_10', 'ISO_A2', 'ISO_A2_EH', 'ISO_A3', 'ISO_A3_EH', 'ISO_N3', 'ISO_N3_EH', 'UN_A3', 'WB_A2', 'WB_A3', 'WOE_ID', 'WOE_ID_EH', 'WOE_NOTE', 'ADM0_ISO', 'ADM0_DIFF', 'ADM0_TLC', 'ADM0_A3_US', 'ADM0_A3_FR', 'ADM0_A3_RU', 'ADM0_A3_ES', 'ADM0_A3_CN', 'ADM0_A3_TW', 'ADM0_A3_IN', 'ADM0_A3_NP', 'ADM0_A3_PK', 'ADM0_A3_DE', 'ADM0_A3_GB', 'ADM0_A3_BR', 'ADM0_A3_IL', 'ADM0_A3_PS', 'ADM0_A3_SA', 'ADM0_A3_EG', 'ADM0_A3_MA', 'ADM0_A3_PT', 'ADM0_A3_AR', 'ADM0_A3_JP', 'ADM0_A3_KO', 'ADM0_A3_VN', 'ADM0_A3_TR', 'AD

In [21]:
# give the income groups simpler names
income_groups = {1: 'H', 2: 'H', 3: 'UM', 4: 'LM', 5: 'L'}
socio['INCOME_GRP'] = socio['INCOME_GRP'].str.split('.').str[0].astype(int)
socio['INCOME_GRP'] = socio['INCOME_GRP'].map(income_groups)


In [22]:
# method 1
rename_columns = {'NAME': 'name', 'CONTINENT': 'continent', 'POP_EST': 'pop_est',
                  'GDP_MD': 'gdp_md_est', 'INCOME_GRP': 'income', 'ISO_A3': 'iso3'}

socio = socio.rename(columns=rename_columns)
socio = socio[SOCIOCOLS].copy()
socio.head()

,pop_est,continent,name,iso3,gdp_md_est,income,geometry
0,889953.0,Oceania,Fiji,FJI,5496,LM,"MULTIPOLYGON (((180 -16.06713, 180 -16.55522, ..."
1,58005463.0,Africa,Tanzania,TZA,63177,L,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3..."
2,603253.0,Africa,W. Sahara,ESH,907,L,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,37589262.0,North America,Canada,CAN,1736425,H,"MULTIPOLYGON (((-122.84 49, -122.97421 49.0025..."
4,328239523.0,North America,United States of America,USA,21433226,H,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312..."


In [23]:
# check for rows with missing values
socio[socio.isnull().any(axis=1)]

,pop_est,continent,name,iso3,gdp_md_est,income,geometry


Finally, we will do a spatial join, to match each blackout location to a country.

In [24]:
# good practice to check crs alignment before geospatial ops
assert socio.crs == blackout.crs

if False: # make True to visualise, a little slow
    fig, ax = plt.subplots(figsize=(10, 4))
    socio.boundary.plot(ax=ax, color='black')
    blackout.plot(ax=ax, color='red', markersize=5)
    fig.suptitle('Blackout locations and country boundaries')

In [25]:
# do the spatial join and save
blackout = gpd.sjoin(blackout, socio, how="left", predicate='within')
blackout = blackout.drop(columns=['index_right', 'geometry'])

blackout.to_csv(os.path.join(WD, "blackout_with_socio.csv"), index=False)

### Appendix: Wind factors
> Scale and shift used by agencies to convert their internal Dvorak 1-minute sustained winds to the officially reported values that are in IBTrACS. From Table 1 in  Knapp, K.R. & Kruk, M.C. (2010): Quantifying Interagency Differences in Tropical Cyclone Best-Track Wind Speed Estimates. Monthly Weather Review 138(4): 1459–1473. https://journals.ametsoc.org/view/journals/mwre/138/4/2009mwr3123.1.xml 

https://climada-python.readthedocs.io/en/stable/_modules/climada/hazard/tc_tracks.html

In [26]:
IBTRACS_AGENCY_1MIN_WIND_FACTOR = {
    "usa": [1.0, 0.0],
    "tokyo": [0.60, 23.3],
    "newdelhi": [1.0, 0.0],
    "reunion": [0.88, 0.0],
    "bom": [0.88, 0.0],
    "nadi": [0.88, 0.0],
    "wellington": [0.88, 0.0],
    'cma': [0.871, 0.0],
    'hko': [0.9, 0.0],
    'ds824': [1.0, 0.0],
    'td9636': [1.0, 0.0],
    'td9635': [1.0, 0.0],
    'neumann': [0.88, 0.0],
    'mlc': [1.0, 0.0],
}
""""""


IBTRACS_USA_AGENCIES = [
    'atcf', 'cphc', 'hurdat_atl', 'hurdat_epa', 'jtwc_cp', 'jtwc_ep', 'jtwc_io',
    'jtwc_sh', 'jtwc_wp', 'nhc_working_bt', 'tcvightals', 'tcvitals'
]

IBTRACS_USA_1MIN_WIND_FACTOR = {
    agency: IBTRACS_AGENCY_1MIN_WIND_FACTOR['usa'] for agency in IBTRACS_USA_AGENCIES
    }
WIND_FACTORS = IBTRACS_AGENCY_1MIN_WIND_FACTOR | IBTRACS_USA_1MIN_WIND_FACTOR

wind_factors = pd.DataFrame(WIND_FACTORS).T
wind_factors.columns = ["scale", "shift"]
wind_factors.index.name = "agency"
wind_factors.to_csv(os.path.join(WD, "wind_factors.csv"))